# TASK 2
Current considerations:
1. Reducing through evolution system -> bad, maybe different function?
3. For subtask 3 the best actual model is the multitask lasso with cross validation, but is slow, 
   Ridge is fast and yields good results on training set, 
   Neuralnet is between the two


To Matt: changed something in the other subtasks and they now perform considerably worse or might not even working.

I marked the sections with "needed" where there is coded needed for running the subtask3. 

It should output the predictions for subtask 3 in pandas dataframe format. I used RidgeCV as it is relatively fast and produces good results on the training set. The best so far is using MultiTaskLassoCV but is considerably slower and the difference in results is not that much.

## Libraries - needed

In [25]:
# Needed libraries
import pandas as pd #Pandas
import numpy as np #Numpy
import sklearn #Sklearn
from sklearn import datasets, linear_model
from sklearn.datasets import make_regression

#libraries needed for preprocessing
from sklearn.preprocessing import StandardScaler

#Libraries needed for imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

#Libraries needed for models
#subtasks 1 and 2
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
#subtask3
from sklearn.linear_model import RidgeCV

#Libraries needed for plotters
from matplotlib import pyplot as plt

#Libraries needed for scoring
import sklearn.metrics as metrics

## Data Manipulation functions - needed

In [26]:
def nan_imputer(nds,method):
    """
    Given a dataset removes NaNs using
    Parameters:
    Input nds - Numpy array: dataset
    Input method - method of imputation to use
    Output nds_xnan - Numpy array: dataset without NaNs
    """
    if method==1:#Sklearn: IterativeImputer, removes NaN considering other features
        imp = IterativeImputer(max_iter=10, random_state=0)
        imp.fit(nds)
        IterativeImputer(random_state=0)
        nds_xnan = imp.transform(nds)
    return nds_xnan

In [27]:
def time_reduction(nds,labels, time, method):
    """
    Given a dataset containing data on consecutive hours outputs a row extracting information time features
    Parameters:
    Input nds - numpy dataset
    Input labels - list labels of dataset
    Input time - time in hours to compress data 
    Input method - method of reduction to use
    Output nds_reduced - dataset compressed 
    """
    nds = pd.DataFrame(nds,columns=labels)
    datalen = len(nds)
    numpatients = datalen / time #number of patients
    
    if method==1:#average of values per patient
        #Reduce by taking mean of columns for each patient
        nds_reduced = nds.groupby('pid',sort=False,as_index=False).mean()
        
    elif method==2:#scoring method based on evolution of patient during stay
        dss = np.array_split(nds,datalen,axis=0) #dataset split for each patient  
        nds_reduced = []
        flagr=True
        for k in range(datalen):#for each patient
            patient = dss[k]#select patient
            npat = patient.to_numpy()
            r_pat = []
            for i in range (np.size(npat,1)):#for each label
                cur_col = npat[:,i]
                temp=0
                ev = 0
                flagn = True
                for j in range(np.size(cur_col)):#for each row
                    this=cur_col[j]
                    if ~(np.isnan(this)):
                        ev = ev + (this-temp)*(j+1) #evolution increasing on time
                        temp=this
                        flagn= False
                if flagn:#if row is all NaN
                    r_pat = np.append(r_pat,np.NaN)#insert NaN
                else:#else
                    r_pat = np.append(r_pat,ev)#insert evolution
            if flagr:#if reduced set is empty
                nds_reduced = np.append(nds_reduced,r_pat)#insert patient
                flagr=False
            else:#if at least one patient has been added
                nds_reduced = np.vstack((nds_reduced, r_pat))#insert patient as row
        #Transform to pandas for compatibility
        nds_reduced=pd.DataFrame(nds_reduced,columns=labels)
    #Reduce considering patient evolution during stay 
    return nds_reduced.to_numpy()

In [28]:
def clean_set(nds,labels,imp_method,time_method,sequence):
    ds_clean = nds
    if sequence:
        ds_clean = time_reduction(ds_clean, labels, 12,time_method)
        ds_clean = nan_imputer(ds_clean,imp_method)
        ds_clean = pd.DataFrame(ds_clean, columns=labels)
    else:
        ds_clean = nan_imputer(ds_clean,imp_method)    
        ds_clean = time_reduction(ds_clean, labels, 12,time_method)
        ds_clean = pd.DataFrame(ds_clean, columns=labels)
    
        
    return ds_clean

## Model choosing functions - needed

In [29]:
def data_fold_10(nds, f_nr, task):
    """
    Given a dataset, outputs two subsets: training set and test set. Test sets is given by the f_nr-th partition of the dataset,
    meanwhile the training set is the remaining of the dataset
    Parameters:
    Input ds - numpy dataset to partition
    Input f_nr - number of the fold that will be the test set 1-10
    Output (testset, trainingset) - tuple containing the test set and training set
    """
    dss = np.array_split(nds,10,axis=0) #dataset split
    testset = dss[f_nr] #test set
    if task==2:
        trainingset = np.hstack(np.delete(dss, f_nr, 0)) #training set 
    else:
        trainingset = np.vstack(np.delete(dss, f_nr, 0)) #training set
    return (testset, trainingset)

In [30]:
def fold10_predict(models, ndsx, ndsy, ndsy_L, task):
    ###Performing 10-fold Cross Validation for each Model
    mlen = len(models) #Number of models
    nscores = np.zeros((mlen,10)) #score of each fold
    for j in range(10):
        #Creating test set and training set from data set 
        if task==2:
            (tes_x, trs_x) = data_fold_10(ndsx, j, task-1)
            (tes_y, trs_y) = data_fold_10(ndsy, j, task)
        else:
            (tes_x, trs_x) = data_fold_10(ndsx, j, task)
            (tes_y, trs_y) = data_fold_10(ndsy, j, task)

        #Perform fitting and predicting for each model
        for i in range(mlen):
            models[i].fit(trs_x,trs_y)
            if task==3:#if task is third we use predict
                tes_yp = models[i].predict(tes_x)

                #Transform into DataFrame for scoring
                df_y = pd.DataFrame(tes_y, columns=ndsy_L)
                df_yp = pd.DataFrame(tes_yp, columns=ndsy_L)
                nscores[i,j] = scores(df_y,df_yp, task)
            else:#else use predict_proba
                tes_yp = models[i].predict_proba(tes_x)

                #Transform into DataFrame for scoring
                df_y = pd.DataFrame(tes_y, columns=ndsy_L)
                if task==2:
                    df_yp = pd.DataFrame(tes_yp[:,0], columns=ndsy_L)
                else:
                    df_yp = pd.DataFrame(tes_yp, columns=ndsy_L)
                nscores[i,j] = scores(df_y,df_yp, task)
    return nscores

In [31]:
def scores(tes_y, tes_yp, task):
    if task==3:
        VITALS = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
        score = np.mean([0.5 + 0.5 * np.maximum(0, metrics.r2_score(tes_y[entry], tes_yp[entry])) for entry in VITALS])
    elif task==2:
        score = metrics.roc_auc_score(tes_y['LABEL_Sepsis'], tes_yp['LABEL_Sepsis'])
    else:
        TESTS = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2']
        score = np.mean([metrics.roc_auc_score(tes_y[entry], tes_yp[entry]) for entry in TESTS])
    
    return score

# Subtasks - needed

In [32]:
#Data needed for all subtasks
#Extracting training labels
dataset_y = pd.read_csv("train_labels.csv")

#Extracting training feature
dataset_x = pd.read_csv("train_features.csv")

#lists that contain labels of dataset
dataset_x_L = list(dataset_x) 

#Standard Scaler of dataset 
scaler = StandardScaler()
scaler.fit(dataset_x)
scaled_data = scaler.transform(dataset_x)
scaled_data = pd.DataFrame(scaled_data,columns=dataset_x_L)

In [33]:
scaled_data

,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,-1.725092,-0.851213,-1.706426,NaN,NaN,-0.558968,NaN,-0.973706,-0.929341,0.116246,...,NaN,0.838622,NaN,1.308314,-1.158184,0.537177,NaN,NaN,0.843444,-0.500542
1,-1.725092,-0.639173,-1.706426,NaN,NaN,NaN,NaN,-0.973706,NaN,NaN,...,NaN,0.838622,NaN,NaN,NaN,0.820569,NaN,NaN,0.113008,-0.500542
2,-1.725092,-0.427133,-1.706426,NaN,NaN,NaN,NaN,-0.973706,NaN,NaN,...,NaN,0.838622,NaN,NaN,NaN,0.423820,NaN,NaN,-0.531494,0.037220
3,-1.725092,-0.215093,-1.706426,NaN,NaN,NaN,NaN,0.168959,NaN,NaN,...,NaN,0.838622,NaN,NaN,NaN,0.197107,NaN,NaN,-0.789295,0.037220
4,-1.725092,-0.003053,-1.706426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.838622,NaN,NaN,-1.539445,-0.199642,NaN,NaN,-0.961162,0.574982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227935,-0.632639,0.208987,1.393535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.256321,NaN,NaN,-0.531494,NaN
227936,-0.632639,0.421027,1.393535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.086286,NaN,NaN,0.027074,NaN
227937,-0.632639,0.633067,1.393535,NaN,NaN,NaN,NaN,-0.973706,NaN,NaN,...,NaN,0.120793,NaN,NaN,NaN,-0.256321,NaN,NaN,0.671577,NaN
227938,-0.632639,0.845107,1.393535,NaN,NaN,NaN,NaN,NaN,-0.206384,NaN,...,NaN,0.120793,NaN,NaN,-0.049060,-0.539713,NaN,NaN,0.113008,NaN


# Subtask1 training models - not needed

In [21]:
#Model set used for training
svcms = SVC(kernel='sigmoid', 
            decision_function_shape='ovr', 
            gamma='auto', 
            probability=True, 
            max_iter=1000)
models1 = []
#1 - Sklearn: OnveVsRestClassifier using svcms
models1 = np.append(models1,OneVsRestClassifier(svcms))
#2 - Sklearn: OneVsOneClassifier using svcms
models1 = np.append(models1,OneVsOneClassifier(svcms))

In [ ]:
#Cleaning dataset in order to remove NaNs and reduce dimensionality
cs1 = clean_set(scaled_data,dataset_x_L,1,1,True)

##Division for the prediction, probabilities divided from the real values
dataset_y1 = dataset_y.loc[:,"LABEL_BaseExcess":"LABEL_EtCO2"] #Labels to be predicted in [0,1] range
ds_p1 = cs1.loc[:,"Time":"pH"] #reduced dataset for prediction, without pid

#labels of datasets
ds_y1_L = list(dataset_y1)
ds_p1_L = list(ds_p1)

#transform into numpy
ndsy1 = dataset_y1.to_numpy()
nds_p1 = ds_p1.to_numpy()

In [ ]:
#Get Predictions
s1 = fold10_predict(models1,nds_p1,ndsy1,ds_y1_L,1)

In [ ]:
pd.DataFrame(s1)

In [ ]:
pd.DataFrame(np.average(s1,axis=1))

# Subtask2 - training models - not needed

In [34]:
#Model list used for predictions
models2 = []
#0 - Sklearn: SVC 
models2 = np.append(models2,SVC(kernel='sigmoid',
                                gamma='auto',
                                probability=True,
                                max_iter=1000))
#1 - Sklearn: SVR
#models2 = np.append(models2,SVR(kernel='sigmoid',
#                                gamma='auto',
#                                max_iter=1000))
#from sklearn.svm import LinearSVC
#models2 = np.append(models2,LinearSVC(random_state=0, tol=1e-5))

In [ ]:
#Preparing dataset used for training
#Clean dataset:
cs2 = clean_set(scaled_data,dataset_x_L,1,1,True)

##Division for the prediction, probabilities divided from the real values
dataset_y2 = dataset_y.loc[:,"LABEL_Sepsis"] #Labels to be predicted in [0,1] range
ds_p2 = cs2.loc[:,"Time":"pH"] #reduced dataset for prediction, without pid

#labels of datasets
ds_y2_L = ["LABEL_Sepsis"]
ds_p2_L = list(ds_p2)

#transform into numpy
ndsy2 = dataset_y2.to_numpy()
nds_p2 = ds_p2.to_numpy()

In [ ]:
#Get predictions
s2 = fold10_predict(models2,nds_p2,ndsy2,ds_y2_L,2)

In [ ]:
pd.DataFrame(s2)

In [ ]:
pd.DataFrame(np.average(s2,axis=1))

# Subtask3 - training models - needed
I included only ridge as it is relatively fast and produces ~0.745

In [35]:
##Model set used for training
models3 = []
#0 - Sklearn: Ridge regression function with alpha 1
models3 = np.append(models3, linear_model.RidgeCV(alphas=[10*a for a in range(1,10)])) 
#1 - Sklearn: Multi Task Lasso Cross Validation on alphas
#models3 = np.append(models3,linear_model.MultiTaskLassoCV(cv=10,
#                                                          alphas=[10**a for a in range(-1,10)],
#                                                          fit_intercept=False,
#                                                          max_iter=1000))
#2 Sklearn: Multitask Elastic Net with Cross Validation
#models3 = np.append(models3,linear_model.MultiTaskElasticNetCV(cv=10,random_state=0))

In [36]:
#Preparing datasets to feed models
##clean dataset using
# scaled_data: dataset_x scaled through StandardScaler
# dataset_x_L: dataset labels
# imp method: 1 - impute with IterativeImputer
# time method: 1 - reduce with average
# sequence: True - first reduce then converge
cs3 = clean_set(scaled_data,dataset_x_L,1,1,True)

##Division for the prediction, probabilities divided from the real values
dataset_y3 = dataset_y.loc[:,"LABEL_RRate":"LABEL_Heartrate"] #Real valued labels
ds_p3 = cs3.loc[:,"Time":"pH"] #reduced dataset for prediction, without pid

#labels of datasets
ds_y3_L = list(dataset_y3)
ds_p3_L = list(ds_p3)

#transform into numpy
ndsy3 = dataset_y3.to_numpy()
nds_p3 = ds_p3.to_numpy()

In [37]:
#Get predictions
s3 = fold10_predict(models3, nds_p3, ndsy3, ds_y3_L, 3)

In [38]:
pd.DataFrame(s3)

,0,1,2,3,4,5,6,7,8,9
0,0.748734,0.746099,0.74074,0.74454,0.732383,0.767828,0.743956,0.732851,0.749092,0.755045


In [39]:
pd.DataFrame(np.average(s3,axis=1))

,0
0,0.746127


In [40]:
#avg_s1 = np.average(s1)
#avg_s2 = np.average(s2)
#avg_s3 = np.average(s3[1])
#np.mean([avg_s1,avg_s2,avg_s3])

# Testset predicition - subtask3 - needed

In [41]:
#This outputs the predictions for subtask3, in pandas format

#extract dataset to predict
testset_x = pd.read_csv("test_features.csv")
testset_x_L = list(testset_x)
test_x = testset_x.to_numpy()

#Standard Scaler of dataset 
scaler = StandardScaler()
scaler.fit(testset_x)
scaled_test = scaler.transform(testset_x)
scaled_test = pd.DataFrame(scaled_test,columns=dataset_x_L)

#cleaning data
test3_x = clean_set(scaled_test,testset_x_L,1,1,True)

ctes3 = test3_x.loc[:,"Time":"pH"]

C:\Users\Omar\Anaconda3\lib\site-packages\sklearn\impute\_iterative.py:603: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [42]:
best_3 = models3[0] #0 is ridge, fast but worse performing
pred3 = best_3.predict(ctes3)
pred3 = pd.DataFrame(pred3,columns=ds_y3_L)

In [43]:
pred3

,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,14.067502,82.084110,98.513927,83.381661
1,18.255210,88.805621,94.957384,102.886702
2,18.769762,83.807880,97.869078,91.823855
3,16.516004,72.584750,95.821337,88.142008
4,19.359560,75.186645,96.004542,61.304297
...,...,...,...,...
12659,19.666170,80.607350,95.685558,103.118070
12660,18.381966,93.726081,98.695124,75.389952
12661,18.685788,70.779083,97.407136,83.841422
12662,16.064031,86.138289,98.432283,97.656868


# Testset prediction not finished

In [ ]:
#Dummy code to combine three results
best_1 = models1[0]#Dummy
best_2 = models2[0]#Dummy
best_3 = models3[1]#Dummy

#extract dataset to predict
testset_x = pd.read_csv("test_features.csv")
testset_x_L = list(testset_x)
test_x = testset_x.to_numpy()
#cleaning data
test1_x = clean_set(test_x,testset_x_L,1,1,True)
test2_x = clean_set(test_x,testset_x_L,1,1,True)
test3_x = clean_set(test_x,testset_x_L,1,1,True)

#reduced dataset for prediction, without pid
ctes1 = test1_x.loc[:,"Time":"pH"] 
ctes2 = test2_x.loc[:,"Time":"pH"]
ctes3 = test3_x.loc[:,"Time":"pH"]

In [ ]:
#prediction using best models for each subtask
pred1 = best_1.predict(ctes1)
pred2 = best_2.predict(ctes2)
pred3 = best_3.predict(ctes3)

In [ ]:
#combine predictions into one dataframe)
#output, 3 digit floats
#pred.to_csv('prediction.zip', index=False, float_format='%.3f', compression='zip')